Initially, we import alll the necessary libraries.

In [381]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from random import shuffle

We import the necessary files and process them.

In [382]:
df_train = pd.read_csv('train_with_label.txt',sep='\t',header=None, error_bad_lines= False)
df_test = pd.read_csv('dev_with_label.txt', sep='\t',header=None, error_bad_lines= False)
df_test_pred =pd.read_csv('test_without_label.txt', sep='\t',header=None, error_bad_lines= False)

b'Skipping line 323: expected 4 fields, saw 5\nSkipping line 823: expected 4 fields, saw 5\nSkipping line 909: expected 4 fields, saw 5\nSkipping line 1013: expected 4 fields, saw 5\nSkipping line 1084: expected 4 fields, saw 5\nSkipping line 1097: expected 4 fields, saw 5\nSkipping line 1141: expected 4 fields, saw 5\nSkipping line 1267: expected 4 fields, saw 5\nSkipping line 1418: expected 4 fields, saw 5\nSkipping line 1486: expected 4 fields, saw 5\nSkipping line 1557: expected 4 fields, saw 5\nSkipping line 1675: expected 4 fields, saw 5\nSkipping line 1747: expected 4 fields, saw 5\nSkipping line 1801: expected 4 fields, saw 5\nSkipping line 1870: expected 4 fields, saw 5\nSkipping line 2025: expected 4 fields, saw 5\nSkipping line 2102: expected 4 fields, saw 5\nSkipping line 2371: expected 4 fields, saw 5\nSkipping line 2799: expected 4 fields, saw 5\nSkipping line 3016: expected 4 fields, saw 5\nSkipping line 3104: expected 4 fields, saw 5\nSkipping line 3317: expected 4 fiel

In [383]:
df_test_pred

,0,1,2
0,test_id_0,"This blackout was largely preventable , said...","This blackout was largely preventable , Ener..."
1,test_id_1,The Denver-based natural gas producer and mark...,The natural gas producer and marketer said the...
2,test_id_2,The other 18 people inside the building - two ...,"The other 18 people in the building , includin..."
3,test_id_3,"Billy Hodge , 11 , is engrossed with the Harry...","Indeed , the Harry Potter series stands in a c..."
4,test_id_4,Families stuck on the highway remained in thei...,Families stuck on the highway were being urged...
...,...,...,...
944,test_id_995,"Sen. Bob Graham , Florida Democrat , raised $ ...","Further back , Sen. Bob Graham of Florida repo..."
945,test_id_996,Micron has declared its first quarterly profit...,Micron 's numbers also marked the first quarte...
946,test_id_997,The momentum in the marketplace continues to ...,The momentum in the marketplace continues to ...
947,test_id_998,"It ends tonight , a grim Keanu Reeves announ...",Keanu Reeves and Hugo Weaving in a scene from ...


In [384]:
#setting column names for test and training set
df_train.columns = ["iter","sent1","sent2","gold_score"]
df_test.columns = ["iter","sent1","sent2","gold_score"]

In [385]:
#remove all rows with null values
df_train = df_train.dropna()
print(df_train.isnull().sum())
df_test = df_test.dropna()
print(df_test.isnull().sum())
print(df_train.shape)
print(df_test.shape)

iter          0
sent1         0
sent2         0
gold_score    0
dtype: int64
iter          0
sent1         0
sent2         0
gold_score    0
dtype: int64
(3850, 4)
(699, 4)


In [386]:
#possible target names 
print("Train data target names: {}".format(df_train["gold_score"].unique()))
print("Test data target names: {}".format(df_test["gold_score"].unique()))

Train data target names: [1. 0.]
Test data target names: [0. 1.]


In [387]:
#Training dataset
df_train['sent1']=df_train['sent1'].astype('string')
df_train['sent2']=df_train['sent2'].astype('string')
df_train['sent1'] = df_train['sent1'].str.lower()
df_train['sent2'] = df_train['sent2'].str.lower()
#Test dataset
df_test['sent1']=df_test['sent1'].astype('string')
df_test['sent2']=df_test['sent2'].astype('string')
df_test['sent1'] = df_test['sent1'].str.lower()
df_test['sent2'] = df_test['sent2'].str.lower()

In [388]:
print("Train data target names: {}".format(df_train["gold_score"].unique()))
print("Test data target names: {}".format(df_test["gold_score"].unique()))

Train data target names: [1. 0.]
Test data target names: [0. 1.]


In [389]:
print(df_train.dtypes)
print(df_test.dtypes)

iter           object
sent1          string
sent2          string
gold_score    float64
dtype: object
iter           object
sent1          string
sent2          string
gold_score    float64
dtype: object


In [390]:
#Training
df_train['sent1'] = df_train['sent1'].str.replace(",","")
df_train['sent2'] = df_train['sent2'].str.replace(",","")
#Testing
df_test['sent1'] = df_test['sent1'].str.replace(",","")
df_test['sent2'] = df_test['sent2'].str.replace(",","")

## Feature 1: WordCount difference

In [391]:
# df_train = df_train.assign(sent1l=lambda x: (x['sent1']))
# df_train = df_train.assign(sent2l=lambda x: (x['sent2']))
# df_test = df_test.assign(sent1l=lambda x: (x['sent1']))
# df_test = df_test.assign(sent2l=lambda x: (x['sent2']))

In [392]:
df_train['sent1l'] = df_train['sent1'].str.split()
df_train['sent2l'] = df_train['sent2'].str.split()
df_test['sent1l'] = df_test['sent1'].str.split()
df_test['sent2l'] = df_test['sent2'].str.split()

In [393]:
df_train['sent1l']=df_train['sent1l'].apply(lambda x: len(x))
df_train['sent2l']=df_train['sent2l'].apply(lambda x: len(x))
df_test['sent1l']=df_test['sent1l'].apply(lambda x: len(x))
df_test['sent2l']=df_test['sent2l'].apply(lambda x: len(x))

In [394]:
df_train = df_train.assign(wcd=lambda x: abs((x['sent1l']-x['sent2l'])))
df_test = df_test.assign(wcd=lambda x: abs((x['sent1l']-x['sent2l'])))

In [395]:
df_test

,iter,sent1,sent2,gold_score,sent1l,sent2l,wcd
0,dev_id_0,local police authorities are treating the expl...,acting new haven police chief francisco ortiz ...,0.0,18,18,0
1,dev_id_1,the report shows that drugs sold in canadian p...,the report shows that drugs sold in canadian p...,1.0,25,30,5
2,dev_id_2,the transition is slated to begin no later tha...,a two-week transition period will begin no lat...,1.0,14,12,2
3,dev_id_3,like viacom ge -- parent of nbc -- is also se...,like viacom general electric is seen as a les...,1.0,25,17,8
4,dev_id_4,last month 62 spanish peacekeepers died when ...,in another disaster 62 spanish peacekeepers w...,1.0,18,21,3
...,...,...,...,...,...,...,...
698,dev_id_719,he is a brother to three-year-old mia from ka...,winslet 28 has a three-year-old daughter mia...,0.0,18,17,1
699,dev_id_720,some 175 million shares traded on the big boar...,some 1.6 billion shares traded on the big boar...,0.0,21,19,2
700,dev_id_721,mr berlusconi is accused of bribing judges to ...,mr berlusconi is accused of bribing judges to ...,1.0,20,22,2
701,dev_id_722,"he added that those "" are not solely american ...",these are not solely american principles nor ...,1.0,17,14,3


## Feature 2: Fuzzywuzzy ration

In [396]:
from fuzzywuzzy import fuzz
import string
fuzz.token_sort_ratio("this is a test", "this is a test!")

100

In [397]:
df_train['fuzz_ratio'] = df_train.apply(lambda row: fuzz.ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test['fuzz_ratio'] = df_test.apply(lambda row: fuzz.ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)


In [398]:
df_train['fuzz_token_sort_ratio'] = df_train.apply(lambda row: fuzz.token_sort_ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test['fuzz_token_sort_ratio'] = df_test.apply(lambda row: fuzz.token_sort_ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)

In [399]:
df_train

,iter,sent1,sent2,gold_score,sent1l,sent2l,wcd,fuzz_ratio,fuzz_token_sort_ratio
0,train_id_0,the democratic candidates also began announcin...,the democratic candidates also began announcin...,1.0,23,25,2,89,88
1,train_id_1,the woman was exposed to the sars virus while ...,the woman was exposed to the sars virus while ...,1.0,33,30,3,98,97
2,train_id_2,he said the problem needs to be corrected befo...,he said the prob lem needs to be corrected bef...,1.0,19,17,2,90,87
3,train_id_3,a representative for phoenix-based u-haul decl...,anthony citrano a representative for whenu d...,0.0,19,13,6,61,61
4,train_id_4,the biggest threat to order seemed to be looti...,the biggest threat to order seemed to be looti...,1.0,25,29,4,79,79
...,...,...,...,...,...,...,...,...,...
3867,train_id_4072,axelrod died in his sleep of heart failure sa...,axelrod died of heart failure while asleep at ...,1.0,14,18,4,72,80
3868,train_id_4073,saddam 's other son odai surrendered friday ...,hussein 's other son uday surrendered yester...,1.0,21,21,0,90,81
3869,train_id_4074,if senator clinton does decide to run in 2008 ...,if mrs clinton does decide to contest the 2008...,1.0,26,27,1,90,88
3870,train_id_4075,the iranian refugee who sewed up his eyes lip...,an iranian kurd who stitched up his eyes lips...,1.0,29,23,6,68,68


## Feature 3: Bleu score

In [400]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [401]:
df_train['bleu_score'] = df_train.apply(lambda row: sentence_bleu(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation)),smoothing_function=SmoothingFunction().method4), axis = 1)
df_test['bleu_score'] = df_test.apply(lambda row: sentence_bleu(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation)),smoothing_function=SmoothingFunction().method4),axis = 1)


In [312]:
df_train

,iter,sent1,sent2,gold_score,sent1l,sent2l,wcd,fuzz_token_sort_ratio,bleu_score
0,train_id_1,the woman was exposed to the sars virus while ...,the woman was exposed to the sars virus while ...,1.0,33,30,3,97,0.004939
1,train_id_2,he said the problem needs to be corrected befo...,he said the prob lem needs to be corrected bef...,1.0,19,17,2,87,0.008011
2,train_id_3,a representative for phoenix-based u-haul decl...,anthony citrano a representative for whenu d...,0.0,19,13,6,61,0.008799
3,train_id_4,the biggest threat to order seemed to be looti...,the biggest threat to order seemed to be looti...,1.0,25,29,4,79,0.004822
4,train_id_5,crews worked to install a new culvert and prep...,crews worked to install a new culvert and repa...,0.0,29,21,8,82,0.006559
...,...,...,...,...,...,...,...,...,...
3866,train_id_4072,axelrod died in his sleep of heart failure sa...,axelrod died of heart failure while asleep at ...,1.0,14,18,4,80,0.006853
3867,train_id_4073,saddam 's other son odai surrendered friday ...,hussein 's other son uday surrendered yester...,1.0,21,21,0,81,0.006430
3868,train_id_4074,if senator clinton does decide to run in 2008 ...,if mrs clinton does decide to contest the 2008...,1.0,26,27,1,88,0.005049
3869,train_id_4075,the iranian refugee who sewed up his eyes lip...,an iranian kurd who stitched up his eyes lips...,1.0,29,23,6,68,0.006053


## Feature 4: Cosine Similarity

In [361]:
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.feature_extraction.text import TfidfVectorizer


# count_vect = CountVectorizer()
# vectorizer = TfidfVectorizer()

# t = cosine_similarity(trsfm[0:1], trsfm)
# print(t[0][1])

# trsfm=vectorizer.fit_transform(corpus)


# corpus = [Document1,Document2]


In [378]:
# trsfm=vectorizer.fit_transform(corpus)
# X_train_counts = df_train.apply(lambda row: count_vect.fit_transform([row['sent1'], row['sent2']]), axis = 1)
# X_train_counts=np.array(X_train_counts)
# print(X_train_counts)

[<2x23 sparse matrix of type '<class 'numpy.int64'>'
 	with 46 stored elements in Compressed Sparse Row format>
 <2x19 sparse matrix of type '<class 'numpy.int64'>'
 	with 31 stored elements in Compressed Sparse Row format>
 <2x19 sparse matrix of type '<class 'numpy.int64'>'
 	with 27 stored elements in Compressed Sparse Row format> ...
 <2x27 sparse matrix of type '<class 'numpy.int64'>'
 	with 49 stored elements in Compressed Sparse Row format>
 <2x32 sparse matrix of type '<class 'numpy.int64'>'
 	with 46 stored elements in Compressed Sparse Row format>
 <2x19 sparse matrix of type '<class 'numpy.int64'>'
 	with 26 stored elements in Compressed Sparse Row format>]


In [380]:
# pd.DataFrame(X_train_counts,columns=count_vect.get_feature_names(),index=['Sentence 1','Sentence 3'])

ValueError: Shape of passed values is (3849, 1), indices imply (2, 19)

In [374]:
# print(X_train_counts)

0         (0, 16)\t3\n  (0, 21)\t1\n  (0, 19)\t2\n  (0...
1         (0, 8)\t1\n  (0, 14)\t1\n  (0, 17)\t2\n  (0,...
2         (0, 14)\t1\n  (0, 7)\t1\n  (0, 12)\t1\n  (0,...
3         (0, 21)\t3\n  (0, 3)\t1\n  (0, 23)\t1\n  (0,...
4         (0, 4)\t1\n  (0, 25)\t1\n  (0, 22)\t2\n  (0,...
                              ...                        
3866      (0, 3)\t2\n  (0, 5)\t1\n  (0, 10)\t1\n  (0, ...
3867      (0, 13)\t1\n  (0, 11)\t1\n  (0, 14)\t1\n  (0...
3868      (0, 14)\t1\n  (0, 21)\t1\n  (0, 6)\t1\n  (0,...
3869      (0, 27)\t2\n  (0, 16)\t1\n  (0, 21)\t1\n  (0...
3870      (0, 8)\t1\n  (0, 15)\t1\n  (0, 7)\t1\n  (0, ...
Length: 3849, dtype: object


In [360]:
# corpus = [Document1,Document3]

# X_train_counts = count_vect.fit_transform(corpus)

# pd.DataFrame(X_train_counts.toarray(),columns=count_vect.get_feature_names(),index=['Document 1','Document 3'])

# vectorizer = TfidfVectorizer()

# trsfm=vectorizer.fit_transform(corpus)
# pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=['Document 1','Document 3'])

# t = cosine_similarity(trsfm[0:1], trsfm)

# print(t[0][1])

0.06435000938807836


In [365]:
# Document3 = """no no fheym sad."""

# corpus = ["Document1","Docume?nt3"]
# X_train_counts = count_vect.fit_transform(corpus)

# pd.DataFrame(X_train_counts.toarray(),columns=count_vect.get_feature_names(),index=['Document 1','Document 3'])

# vectorizer = TfidfVectorizer()

# trsfm=vectorizer.fit_transform(corpus)
# pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=['Document 1','Document 3'])

# print(cosine_similarity(trsfm[0:1], trsfm))

In [341]:
# df_train_vectors = df_train.apply(lambda row: vectorizer.fit_transform(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
# df_test_vectors = df_test.apply(lambda row: vectorizer.fit_transform(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)


## Reshaping

In [313]:
print(df_train.dtypes)
print(df_test.dtypes)

iter                      object
sent1                     string
sent2                     string
gold_score               float64
sent1l                     int64
sent2l                     int64
wcd                        int64
fuzz_token_sort_ratio      int64
bleu_score               float64
dtype: object
iter                      object
sent1                     string
sent2                     string
gold_score               float64
sent1l                     int64
sent2l                     int64
wcd                        int64
fuzz_token_sort_ratio      int64
bleu_score               float64
dtype: object


In [323]:
X_train = df_train.drop(columns=['iter','sent1','sent2','gold_score','sent1l','sent2l']).values
X_test = df_test.drop(columns=['iter','sent1','sent2','gold_score','sent1l','sent2l']).values
y_train = df_train['gold_score'].values
y_test =df_test['gold_score'].values

normalizer = StandardScaler()
# X_train_val = X_train_val.reshape(-1,1)
# X_test = X_test.reshape(-1,1)
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)
print("train_val: {}, test: {}".format(X_train.shape, X_test.shape))


train_val: (3849, 4), test: (698, 4)


## Logistic Regression model

In [331]:
# retrain the model
clf = LogisticRegression(penalty='l2', solver='lbfgs')
clf.fit(X_train_val, y_train_val)

# evaluate the model on the testing set
y_test_pred = clf.predict(X_test)


acc = accuracy_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)


print("accuracy: {:.3f}, recall: {:.3f}, precision: {:.3f}, f1: {:.3f},".format(acc, recall, precision, f1))

accuracy: 0.622, recall: 0.926, precision: 0.577, f1: 0.711,


## SVM Model

In [332]:
# retrain the model
clf = svm.SVC()
clf.fit(X_train, y_train)

# evaluate the model on the testing set
y_test_pred = clf.predict(X_test)


acc = accuracy_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)


print("accuracy: {:.3f}, recall: {:.3f}, precision: {:.3f}, f1: {:.3f},".format(acc, recall, precision, f1))

accuracy: 0.623, recall: 0.946, precision: 0.576, f1: 0.716,


In [ ]:
folds = 3

num_train_val = X_train.shape[0] 

# shuffle the index of samples in the train_val set
index_of_samples = np.arange(num_train_val) 
shuffle(index_of_samples)

# split the index of the train_valid set into 5 folds
index_of_folds = index_of_samples.reshape(folds, -1)
print(index_of_folds)

regularization_coefficient = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10.0,20.0,50.0,100.0]

best_acc = 0.0
best_reg = 0.0

for reg in regularization_coefficient:
    
    # 3-fold cross-validation
    sum_acc = 0.0
    for fold in range(folds):
        
        index_of_folds_temp = index_of_folds.copy()
        
        valid_index = index_of_folds_temp[fold,:].reshape(-1) #get the index of the validation set
        train_index = np.delete(index_of_folds_temp, fold, 0).reshape(-1) #get the index of the training set
        
#         # training set
        X_train = X_train[train_index]
        y_train = y_train[train_index]
        
        # validation set
        X_valid = X_train[valid_index]
        y_valid = y_train[valid_index]
                
        # build the model with different hyperparameters
        clf = LogisticRegression(penalty='l2', C=reg, solver='lbfgs')
        
        #train the model with the training set
        clf.fit(X_train, y_train)
        
        y_valid_pred = clf.predict(X_valid)
        acc = accuracy_score(y_valid, y_valid_pred)
        
        sum_acc += acc
    
    cur_acc = sum_acc / folds
    
    print("reg_coeff: {}, acc: {:.3f}".format(1.0/reg, cur_acc))
    
    # store the best hyperparameter
    if cur_acc > best_acc:
        best_acc = cur_acc
        best_reg = reg

Pre-processing: Remove double quotes that are not in pair. If possible create pairs correctly
df = pd.read_csv('data_reading_test.txt', sep='\t', header=None)

Open and load the dataset after preprocessing

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.50, shuffle=False)
print("Train data target names: {}".format(df_train[3].unique()))

print('Size of training sample: {}'.format(len(df_train)))
print('Size of testting samples: {}'.format(len(df_test)))
print(df_train.shape)

#TF-IDF representation for each document
vectorizer = TfidfVectorizer(stop_words='english')
# to_vectorize = df_train.drop(columns=[0,3])
# print(to_vectorize)
# test = vectorizer.fit(to_vectorize2)
# print(test)
# data_train_vectors1 = vectorizer.fit_transform(df_train[1])
# data_train_vectors2 = vectorizer.fit_transform(df_train[2])
# data_test_vectors = vectorizer.transform(df_test[1])

# print(data_train_vectors.shape, data_test_vectors.shape)
df_train_1 = vectorizer.fit_transform(df_train[1])
df_train_2 = vectorizer.fit_transform(df_train[2])
df_test_1 = vectorizer.fit_transform(df_test[1])
df_test_2 = vectorizer.fit_transform(df_test[2])
# df_train[1] = df_train[1].replace(to_replace="\t", value=" ", regex=True)
# df_test =  vectorizer.fit_transform(df[2])
print(df_train_1.shape)
print(df_train_2.shape)
print(df_test_1.shape)
print(df_test_2.shape)
# print(df_train_1)